### SVM

Implemantação do classificador utilizando SVM.

In [1]:
import nltk
import itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from string import ascii_lowercase

from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report

from pymongo import MongoClient

%matplotlib inline

In [2]:
'''
Importação dos dados da base
'''
client = MongoClient()
client = MongoClient('localhost', 27800)
db = client.network_logs
db_unique_hostnames = db.unique_hostnames

df_hostnames = pd.DataFrame(list(db_unique_hostnames.find({})))

print(len(df_hostnames))
df_hostnames.head()

24208


,_id,catdesc,category,hostname,normalized_hostname,normalized_hostname_url,normalized_url,service,url
0,59a1e31988b032071ef8c495,Information Technology,Information Technology,sb.scorecardresearch.com,scorecardresearch,scorecardresearch,,HTTPS,/
1,59a1e31a88b032071ef8c496,Information Technology,Information Technology,test.com,test,testpartneruidUPcbedebee,partneruidUPcbedebee,HTTP,/?partner_uid=UP47c5b905-0e1d-11e7-b092-00163e...
2,59a1e31a88b032071ef8c497,Information Technology,Information Technology,nexus-websocket-b.intercom.io,nexuswebsocketbintercom,nexuswebsocketbintercom,,HTTPS,/
3,59a1e31a88b032071ef8c498,Malicious Websites,Malicious or Illegal,aorta.clickagy.com,aortaclickagy,aortaclickagypixelgifadvertiseridgtqxdqpglista...,pixelgifadvertiseridgtqxdqpglistaedjpnobncchcm...,HTTP,/pixel.gif?advertiser_id=1gt8qx97dqpg&list=11a...
4,59a1e31a88b032071ef8c499,Information Technology,Information Technology,usersync.videoamp.com,usersyncvideoamp,usersyncvideoampusersyncpartneridpartneruserid...,usersyncpartneridpartneruseridredirecthttpsync...,HTTP,/usersync?partner_id=5797361&partner_user_id=4...


In [3]:
'''
Criação do vocabulário
'''
keywords = [''.join(trig) for trig in itertools.product(ascii_lowercase, repeat = 3)]
len(keywords)

17576

### Vetorização e frequência dos termos

In [4]:
count_vectorizer = CountVectorizer(analyzer='char', ngram_range=(3,3), vocabulary=keywords)
words_vector = count_vectorizer.transform(df_hostnames['normalized_hostname'])
tf_transformer = TfidfTransformer(use_idf=False).fit(words_vector)
urls_tf = tf_transformer.transform(words_vector)

### Separação do conjunto

In [5]:
labels = df_hostnames['category']
url_train,url_test,label_train,label_test = train_test_split(urls_tf, labels, test_size=0.3)

### Classificação e teste

In [6]:
classifier_model = LinearSVC().fit(url_train, label_train)

In [7]:
predictions = classifier_model.predict(url_test)

### Relatório de classificação

In [8]:
print(classification_report(label_test, predictions))

                                 precision    recall  f1-score   support

                          Adult       0.00      0.00      0.00        10
                    Advertising       0.57      0.57      0.57       320
               Arts and Culture       0.44      0.26      0.33        42
                       Business       0.36      0.43      0.39       697
                Content Servers       0.83      0.71      0.76       362
                 Domain Parking       0.00      0.00      0.00        16
                      Education       0.62      0.58      0.60       225
                  Entertainment       0.36      0.19      0.25       132
       File Sharing and Storage       0.88      0.80      0.84       130
            Finance and Banking       0.59      0.41      0.48        73
Freeware and Software Downloads       0.56      0.34      0.43        67
                          Games       0.69      0.63      0.66       176
            Health and Wellness       0.46      0.

### GridSearch

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
#np.linspace(0.001, 10, num=10)

In [11]:
'''
Parâmetros para GridSearch
'''

#Gerar com linspace
C_range = np.linspace(0.001, 10, num=10)

#np.logspace(-1, 10, num=13)                     
#np.linspace(0.001, 10, num=20)
gamma_range = np.logspace(-9, 3, 13)

print(C_range)
#print(gamma_range)

param_grid = dict(C=C_range, gamma=gamma_range)

[  1.00000000e-03   1.11200000e+00   2.22300000e+00   3.33400000e+00
   4.44500000e+00   5.55600000e+00   6.66700000e+00   7.77800000e+00
   8.88900000e+00   1.00000000e+01]


In [12]:
grid = GridSearchCV(SVC(max_iter=1000, cache_size=800),param_grid,refit=True,verbose=3, n_jobs=4)

In [13]:
# May take awhile!
grid.fit(url_train, label_train)

Fitting 3 folds for each of 130 candidates, totalling 390 fits
[CV] C=0.001, gamma=1e-09 ............................................
[CV] C=0.001, gamma=1e-09 ............................................
[CV] C=0.001, gamma=1e-09 ............................................
[CV] C=0.001, gamma=1e-08 ............................................


/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Converge

[CV] ... C=0.001, gamma=1e-09, score=0.2702654867256637, total= 1.0min
[CV] C=0.001, gamma=1e-08 ............................................
[CV] .. C=0.001, gamma=1e-09, score=0.28335699077359827, total= 1.0min
[CV] C=0.001, gamma=1e-08 ............................................
[CV] ... C=0.001, gamma=1e-09, score=0.2647110796960594, total= 1.0min
[CV] C=0.001, gamma=1e-07 ............................................
[CV] ... C=0.001, gamma=1e-08, score=0.2647110796960594, total= 1.0min
[CV] C=0.001, gamma=1e-07 ............................................


/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, Converge

[CV] .. C=0.001, gamma=1e-08, score=0.28335699077359827, total=  51.3s
[CV] C=0.001, gamma=1e-07 ............................................
[CV] ... C=0.001, gamma=1e-08, score=0.2702654867256637, total=  51.9s
[CV] C=0.001, gamma=1e-06 ............................................
[CV] ... C=0.001, gamma=1e-07, score=0.3238938053097345, total=  51.2s
[CV] C=0.001, gamma=1e-06 ............................................
[CV] ... C=0.001, gamma=1e-07, score=0.3212581728220534, total=  51.6s
[CV] C=0.001, gamma=1e-06 ............................................


...

[CV] ... C=10.0, gamma=100.0, score=0.13165365507452093, total= 1.8min
[CV] .. C=10.0, gamma=1000.0, score=0.12475702420922424, total= 1.8min
[CV] ... C=10.0, gamma=1000.0, score=0.1088495575221239, total= 1.8min


/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


[CV] ... C=10.0, gamma=1000.0, score=0.1717530163236338, total= 1.6min


[Parallel(n_jobs=4)]: Done 390 out of 390 | elapsed: 177.0min finished
/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=800, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=1000, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'C': array([  1.00000e-03,   1.11200e+00,   2.22300e+00,   3.33400e+00,
         4.44500e+00,   5.55600e+00,   6.66700e+00,   7.77800e+00,
         8.88900e+00,   1.00000e+01]), 'gamma': array([  1.00000e-09,   1.00000e-08,   1.00000e-07,   1.00000e-06,
         1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [14]:
grid.best_params_

{'C': 4.4450000000000003, 'gamma': 1.0}

In [15]:
grid.best_estimator_

SVC(C=4.4450000000000003, cache_size=800, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1.0, kernel='rbf',
  max_iter=1000, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
grid_predictions = grid.predict(url_test)

In [17]:
print(classification_report(label_test,grid_predictions))

                                 precision    recall  f1-score   support

                          Adult       0.00      0.00      0.00        10
                    Advertising       0.66      0.56      0.61       320
               Arts and Culture       0.69      0.26      0.38        42
                       Business       0.28      0.60      0.38       697
                Content Servers       0.85      0.72      0.78       362
                 Domain Parking       0.00      0.00      0.00        16
                      Education       0.75      0.52      0.61       225
                  Entertainment       0.59      0.23      0.33       132
       File Sharing and Storage       0.97      0.81      0.88       130
            Finance and Banking       0.73      0.33      0.45        73
Freeware and Software Downloads       0.84      0.39      0.53        67
                          Games       0.81      0.60      0.69       176
            Health and Wellness       0.69      0.

/Users/luanrubensf/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
